In [1]:
import os
from glob import glob

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

from torch.utils.data import Dataset, DataLoader

import numpy as np
from PIL import Image

In [13]:
seed = 1

lr = 0.001
momentum = 0.5

batch_size = 64
test_batch_size = 64

epochs = 1000
no_cuda = False
log_interval = 100

# Model

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

# Preprocess

In [4]:
train_paths = glob('dataset/mnist_png/training/*/*.png')[:1000]
test_paths = glob('dataset/mnist_png/testing/*/*.png')[:1000]

len(train_paths), len(test_paths)

(1000, 1000)

참고: https://pytorch.org/tutorials/beginner/data_loading_tutorial.html#dataset-class

In [5]:
class Dataset(Dataset):
    def __init__(self, data_paths, transform=None):

        self.data_paths = data_paths
        self.transform = transform

    def __len__(self):
        return len(self.data_paths)

    def __getitem__(self, idx):
        path = self.data_paths[idx]
        image = Image.open(path).convert("L")
        label = int(path.split('\\')[-2])
        
        if self.transform:
            image = self.transform(image)

        return image, label

In [6]:
torch.manual_seed(seed)

use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 0, 'pin_memory': True} if use_cuda else {}

train_loader = torch.utils.data.DataLoader(
    Dataset(train_paths, 
            transforms.Compose([
                transforms.RandomHorizontalFlip(), 
                transforms.ToTensor(), 
                transforms.Normalize(
                    mean=[0.406], 
                    std=[0.225])])
           ),
    batch_size=batch_size, 
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    Dataset(test_paths,
           transforms.Compose([
               transforms.ToTensor(), 
               transforms.Normalize(
                   mean=[0.406], 
                   std=[0.225])])
           ),
    batch_size=batch_size, 
    shuffle=False
)

# Optimization

In [7]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

# Training

In [8]:
import torchvision
from torch.utils.tensorboard import SummaryWriter

In [9]:
writer = SummaryWriter()

### example
```python
image, label = next(iter(train_loader))

grid = torchvision.utils.make_grid(image)
writer.add_image('images', grid, epoch)
```

In [14]:
for epoch in range(1, epochs + 1):
    # Train Mode
    model.train()

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)  # https://pytorch.org/docs/stable/nn.html#nll-loss
        loss.backward()
        optimizer.step()

        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    
    # Test mode
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    
    accuracy = 100. * correct / len(test_loader.dataset)
    
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        accuracy))
    
    if epoch == 0:
        grid = torchvision.utils.make_grid(data)
        writer.add_image('images', grid, epoch)
        writer.add_graph(model, data)
        
    writer.add_scalar('Loss/train/', loss, epoch)
    writer.add_scalar('Loss/test/', test_loss, epoch)
    writer.add_scalar('Accuracy/test/', accuracy, epoch)

Train Epoch: 1 [0/1000 (0%)]	Loss: 0.000068

Test set: Average loss: 0.1839, Accuracy: 980/1000 (98%)

Train Epoch: 2 [0/1000 (0%)]	Loss: 0.000076

Test set: Average loss: 0.1840, Accuracy: 980/1000 (98%)

Train Epoch: 3 [0/1000 (0%)]	Loss: 0.000093

Test set: Average loss: 0.1841, Accuracy: 980/1000 (98%)

Train Epoch: 4 [0/1000 (0%)]	Loss: 0.000088

Test set: Average loss: 0.1841, Accuracy: 980/1000 (98%)

Train Epoch: 5 [0/1000 (0%)]	Loss: 0.000085

Test set: Average loss: 0.1842, Accuracy: 980/1000 (98%)

Train Epoch: 6 [0/1000 (0%)]	Loss: 0.000071

Test set: Average loss: 0.1842, Accuracy: 980/1000 (98%)

Train Epoch: 7 [0/1000 (0%)]	Loss: 0.000077

Test set: Average loss: 0.1843, Accuracy: 980/1000 (98%)

Train Epoch: 8 [0/1000 (0%)]	Loss: 0.000076

Test set: Average loss: 0.1844, Accuracy: 980/1000 (98%)

Train Epoch: 9 [0/1000 (0%)]	Loss: 0.000078

Test set: Average loss: 0.1844, Accuracy: 980/1000 (98%)

Train Epoch: 10 [0/1000 (0%)]	Loss: 0.000071

Test set: Average loss: 0.1


Test set: Average loss: 0.1885, Accuracy: 980/1000 (98%)

Train Epoch: 81 [0/1000 (0%)]	Loss: 0.000057

Test set: Average loss: 0.1886, Accuracy: 980/1000 (98%)

Train Epoch: 82 [0/1000 (0%)]	Loss: 0.000060

Test set: Average loss: 0.1886, Accuracy: 980/1000 (98%)

Train Epoch: 83 [0/1000 (0%)]	Loss: 0.000059

Test set: Average loss: 0.1887, Accuracy: 980/1000 (98%)

Train Epoch: 84 [0/1000 (0%)]	Loss: 0.000061

Test set: Average loss: 0.1887, Accuracy: 980/1000 (98%)

Train Epoch: 85 [0/1000 (0%)]	Loss: 0.000060

Test set: Average loss: 0.1888, Accuracy: 980/1000 (98%)

Train Epoch: 86 [0/1000 (0%)]	Loss: 0.000070

Test set: Average loss: 0.1888, Accuracy: 980/1000 (98%)

Train Epoch: 87 [0/1000 (0%)]	Loss: 0.000065

Test set: Average loss: 0.1889, Accuracy: 980/1000 (98%)

Train Epoch: 88 [0/1000 (0%)]	Loss: 0.000066

Test set: Average loss: 0.1889, Accuracy: 980/1000 (98%)

Train Epoch: 89 [0/1000 (0%)]	Loss: 0.000059

Test set: Average loss: 0.1890, Accuracy: 980/1000 (98%)

Train


Test set: Average loss: 0.1923, Accuracy: 980/1000 (98%)

Train Epoch: 160 [0/1000 (0%)]	Loss: 0.000056

Test set: Average loss: 0.1923, Accuracy: 980/1000 (98%)

Train Epoch: 161 [0/1000 (0%)]	Loss: 0.000048

Test set: Average loss: 0.1924, Accuracy: 980/1000 (98%)

Train Epoch: 162 [0/1000 (0%)]	Loss: 0.000068

Test set: Average loss: 0.1924, Accuracy: 980/1000 (98%)

Train Epoch: 163 [0/1000 (0%)]	Loss: 0.000051

Test set: Average loss: 0.1924, Accuracy: 980/1000 (98%)

Train Epoch: 164 [0/1000 (0%)]	Loss: 0.000047

Test set: Average loss: 0.1925, Accuracy: 980/1000 (98%)

Train Epoch: 165 [0/1000 (0%)]	Loss: 0.000048

Test set: Average loss: 0.1925, Accuracy: 980/1000 (98%)

Train Epoch: 166 [0/1000 (0%)]	Loss: 0.000055

Test set: Average loss: 0.1926, Accuracy: 980/1000 (98%)

Train Epoch: 167 [0/1000 (0%)]	Loss: 0.000051

Test set: Average loss: 0.1926, Accuracy: 980/1000 (98%)

Train Epoch: 168 [0/1000 (0%)]	Loss: 0.000063

Test set: Average loss: 0.1927, Accuracy: 980/1000 (98


Test set: Average loss: 0.1954, Accuracy: 980/1000 (98%)

Train Epoch: 239 [0/1000 (0%)]	Loss: 0.000040

Test set: Average loss: 0.1955, Accuracy: 980/1000 (98%)

Train Epoch: 240 [0/1000 (0%)]	Loss: 0.000044

Test set: Average loss: 0.1955, Accuracy: 980/1000 (98%)

Train Epoch: 241 [0/1000 (0%)]	Loss: 0.000046

Test set: Average loss: 0.1955, Accuracy: 980/1000 (98%)

Train Epoch: 242 [0/1000 (0%)]	Loss: 0.000044

Test set: Average loss: 0.1956, Accuracy: 980/1000 (98%)

Train Epoch: 243 [0/1000 (0%)]	Loss: 0.000040

Test set: Average loss: 0.1956, Accuracy: 980/1000 (98%)

Train Epoch: 244 [0/1000 (0%)]	Loss: 0.000043

Test set: Average loss: 0.1956, Accuracy: 980/1000 (98%)

Train Epoch: 245 [0/1000 (0%)]	Loss: 0.000037

Test set: Average loss: 0.1957, Accuracy: 980/1000 (98%)

Train Epoch: 246 [0/1000 (0%)]	Loss: 0.000039

Test set: Average loss: 0.1957, Accuracy: 980/1000 (98%)

Train Epoch: 247 [0/1000 (0%)]	Loss: 0.000039

Test set: Average loss: 0.1957, Accuracy: 980/1000 (98


Test set: Average loss: 0.1981, Accuracy: 980/1000 (98%)

Train Epoch: 318 [0/1000 (0%)]	Loss: 0.000036

Test set: Average loss: 0.1982, Accuracy: 980/1000 (98%)

Train Epoch: 319 [0/1000 (0%)]	Loss: 0.000040

Test set: Average loss: 0.1982, Accuracy: 980/1000 (98%)

Train Epoch: 320 [0/1000 (0%)]	Loss: 0.000042

Test set: Average loss: 0.1982, Accuracy: 980/1000 (98%)

Train Epoch: 321 [0/1000 (0%)]	Loss: 0.000042

Test set: Average loss: 0.1983, Accuracy: 980/1000 (98%)

Train Epoch: 322 [0/1000 (0%)]	Loss: 0.000036

Test set: Average loss: 0.1983, Accuracy: 980/1000 (98%)

Train Epoch: 323 [0/1000 (0%)]	Loss: 0.000041

Test set: Average loss: 0.1983, Accuracy: 980/1000 (98%)

Train Epoch: 324 [0/1000 (0%)]	Loss: 0.000042

Test set: Average loss: 0.1984, Accuracy: 980/1000 (98%)

Train Epoch: 325 [0/1000 (0%)]	Loss: 0.000032

Test set: Average loss: 0.1984, Accuracy: 980/1000 (98%)

Train Epoch: 326 [0/1000 (0%)]	Loss: 0.000039

Test set: Average loss: 0.1984, Accuracy: 980/1000 (98

...


Test set: Average loss: 0.2063, Accuracy: 980/1000 (98%)

Train Epoch: 634 [0/1000 (0%)]	Loss: 0.000022

Test set: Average loss: 0.2064, Accuracy: 980/1000 (98%)

Train Epoch: 635 [0/1000 (0%)]	Loss: 0.000024

Test set: Average loss: 0.2064, Accuracy: 980/1000 (98%)

Train Epoch: 636 [0/1000 (0%)]	Loss: 0.000025

Test set: Average loss: 0.2064, Accuracy: 980/1000 (98%)

Train Epoch: 637 [0/1000 (0%)]	Loss: 0.000027

Test set: Average loss: 0.2064, Accuracy: 980/1000 (98%)

Train Epoch: 638 [0/1000 (0%)]	Loss: 0.000023

Test set: Average loss: 0.2064, Accuracy: 980/1000 (98%)

Train Epoch: 639 [0/1000 (0%)]	Loss: 0.000024

Test set: Average loss: 0.2065, Accuracy: 980/1000 (98%)

Train Epoch: 640 [0/1000 (0%)]	Loss: 0.000027

Test set: Average loss: 0.2065, Accuracy: 980/1000 (98%)

Train Epoch: 641 [0/1000 (0%)]	Loss: 0.000024

Test set: Average loss: 0.2065, Accuracy: 980/1000 (98%)

Train Epoch: 642 [0/1000 (0%)]	Loss: 0.000022

Test set: Average loss: 0.2065, Accuracy: 980/1000 (98


Test set: Average loss: 0.2080, Accuracy: 980/1000 (98%)

Train Epoch: 713 [0/1000 (0%)]	Loss: 0.000021

Test set: Average loss: 0.2080, Accuracy: 980/1000 (98%)

Train Epoch: 714 [0/1000 (0%)]	Loss: 0.000022

Test set: Average loss: 0.2080, Accuracy: 980/1000 (98%)

Train Epoch: 715 [0/1000 (0%)]	Loss: 0.000018

Test set: Average loss: 0.2080, Accuracy: 980/1000 (98%)

Train Epoch: 716 [0/1000 (0%)]	Loss: 0.000019

Test set: Average loss: 0.2080, Accuracy: 980/1000 (98%)

Train Epoch: 717 [0/1000 (0%)]	Loss: 0.000019

Test set: Average loss: 0.2081, Accuracy: 980/1000 (98%)

Train Epoch: 718 [0/1000 (0%)]	Loss: 0.000020

Test set: Average loss: 0.2081, Accuracy: 980/1000 (98%)

Train Epoch: 719 [0/1000 (0%)]	Loss: 0.000022

Test set: Average loss: 0.2081, Accuracy: 980/1000 (98%)

Train Epoch: 720 [0/1000 (0%)]	Loss: 0.000020

Test set: Average loss: 0.2081, Accuracy: 980/1000 (98%)

Train Epoch: 721 [0/1000 (0%)]	Loss: 0.000024

Test set: Average loss: 0.2081, Accuracy: 980/1000 (98


Test set: Average loss: 0.2094, Accuracy: 980/1000 (98%)

Train Epoch: 792 [0/1000 (0%)]	Loss: 0.000017

Test set: Average loss: 0.2095, Accuracy: 980/1000 (98%)

Train Epoch: 793 [0/1000 (0%)]	Loss: 0.000022

Test set: Average loss: 0.2095, Accuracy: 980/1000 (98%)

Train Epoch: 794 [0/1000 (0%)]	Loss: 0.000020

Test set: Average loss: 0.2095, Accuracy: 980/1000 (98%)

Train Epoch: 795 [0/1000 (0%)]	Loss: 0.000020

Test set: Average loss: 0.2095, Accuracy: 980/1000 (98%)

Train Epoch: 796 [0/1000 (0%)]	Loss: 0.000019

Test set: Average loss: 0.2095, Accuracy: 980/1000 (98%)

Train Epoch: 797 [0/1000 (0%)]	Loss: 0.000021

Test set: Average loss: 0.2096, Accuracy: 980/1000 (98%)

Train Epoch: 798 [0/1000 (0%)]	Loss: 0.000021

Test set: Average loss: 0.2096, Accuracy: 980/1000 (98%)

Train Epoch: 799 [0/1000 (0%)]	Loss: 0.000021

Test set: Average loss: 0.2096, Accuracy: 980/1000 (98%)

Train Epoch: 800 [0/1000 (0%)]	Loss: 0.000020

Test set: Average loss: 0.2096, Accuracy: 980/1000 (98


Test set: Average loss: 0.2108, Accuracy: 980/1000 (98%)

Train Epoch: 871 [0/1000 (0%)]	Loss: 0.000017

Test set: Average loss: 0.2109, Accuracy: 980/1000 (98%)

Train Epoch: 872 [0/1000 (0%)]	Loss: 0.000020

Test set: Average loss: 0.2109, Accuracy: 980/1000 (98%)

Train Epoch: 873 [0/1000 (0%)]	Loss: 0.000016

Test set: Average loss: 0.2109, Accuracy: 980/1000 (98%)

Train Epoch: 874 [0/1000 (0%)]	Loss: 0.000020

Test set: Average loss: 0.2109, Accuracy: 980/1000 (98%)

Train Epoch: 875 [0/1000 (0%)]	Loss: 0.000017

Test set: Average loss: 0.2109, Accuracy: 980/1000 (98%)

Train Epoch: 876 [0/1000 (0%)]	Loss: 0.000017

Test set: Average loss: 0.2109, Accuracy: 980/1000 (98%)

Train Epoch: 877 [0/1000 (0%)]	Loss: 0.000022

Test set: Average loss: 0.2110, Accuracy: 980/1000 (98%)

Train Epoch: 878 [0/1000 (0%)]	Loss: 0.000018

Test set: Average loss: 0.2110, Accuracy: 980/1000 (98%)

Train Epoch: 879 [0/1000 (0%)]	Loss: 0.000018

Test set: Average loss: 0.2110, Accuracy: 980/1000 (98


Test set: Average loss: 0.2121, Accuracy: 980/1000 (98%)

Train Epoch: 950 [0/1000 (0%)]	Loss: 0.000019

Test set: Average loss: 0.2121, Accuracy: 980/1000 (98%)

Train Epoch: 951 [0/1000 (0%)]	Loss: 0.000017

Test set: Average loss: 0.2122, Accuracy: 980/1000 (98%)

Train Epoch: 952 [0/1000 (0%)]	Loss: 0.000018

Test set: Average loss: 0.2122, Accuracy: 980/1000 (98%)

Train Epoch: 953 [0/1000 (0%)]	Loss: 0.000017

Test set: Average loss: 0.2122, Accuracy: 980/1000 (98%)

Train Epoch: 954 [0/1000 (0%)]	Loss: 0.000016

Test set: Average loss: 0.2122, Accuracy: 980/1000 (98%)

Train Epoch: 955 [0/1000 (0%)]	Loss: 0.000014

Test set: Average loss: 0.2122, Accuracy: 980/1000 (98%)

Train Epoch: 956 [0/1000 (0%)]	Loss: 0.000017

Test set: Average loss: 0.2122, Accuracy: 980/1000 (98%)

Train Epoch: 957 [0/1000 (0%)]	Loss: 0.000018

Test set: Average loss: 0.2123, Accuracy: 980/1000 (98%)

Train Epoch: 958 [0/1000 (0%)]	Loss: 0.000017

Test set: Average loss: 0.2123, Accuracy: 980/1000 (98